In [4]:
'''
实例化一个小型卷积神经网络
'''
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

2021-12-13 17:51:37.137234: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
Total params: 55,744
Trainable params: 55,744
Non-traina

关于Conv2的的具体内容，可以查看[官方文档](https://keras.io/zh/layers/convolutional/#conv2d)

In [6]:
# 将3D张量转换为1D张量
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
 flatten (Flatten)           (None, 576)               0

In [11]:
# 在MNIST上训练卷积神经网络
from keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
938/938 [==============================] - 19s 20ms/step - loss: 0.1784 - accuracy: 0.9441
Epoch 2/5
938/938 [==============================] - 21s 22ms/step - loss: 0.0477 - accuracy: 0.9854
Epoch 3/5
938/938 [==============================] - 21s 22ms/step - loss: 0.0326 - accuracy: 0.9899
Epoch 4/5
938/938 [==============================] - 21s 22ms/step - loss: 0.0251 - accuracy: 0.9925
Epoch 5/5
938/938 [==============================] - 20s 22ms/step - loss: 0.0206 - accuracy: 0.9934


In [12]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

313/313 [==============================] - 1s 3ms/step - loss: 0.0259 - accuracy: 0.9923


In [13]:
print('test_acc:', test_acc)
print('test_loss:', test_loss)

test_acc: 0.9922999739646912
test_loss: 0.025934725999832153


从运行结果可以看出，正确率是99%，比之前的正确率要高（当然不建议太高），但是花费时间明显比之前要长。这主要是卷积涉及到大量计算，而现在是跑在cpu上的，如果是在GPU上应该会更快。

我自己在google的colab上用gpu跑了下，每个epoch用时 9s左右，而在自己21年最强款macmini的cpu上跑，耗时22s。差距还是比较大的，而且google的colab还是多人共享，分配的资源也不是很多。所以用卷积网络最好是跑在GPU上。
![img](images/005-001.png)

## 卷积运算
为什么这个简单的卷积神经网络效果这么好？因为之前密集连接层采用的是全局模式，而卷积层学到的是局部模式。

因为具有2个特质：
- 平移不变形(translation invariant),比如某个特征电脑在右下角发现后，在左下角发现的时候也能辨认出来，而密集连接层，必须重新学习全局。
- 学到模式的空间层次结构。先学会直线，后学会曲线，学会耳朵，越来越抽象的概念认知。

以上这2个特性都是人类认知事物的自然过程。

原本是一张5x5的特征图，卷积后输出的特征图是3x3
![img](images/005-002.png)

如果希望输出的特征图空间维度与输入相同，可以使用填充(在之前提到过了),结果如下：
![img](images/005-003.png)

keras中的conv2d卷积层中有个padding模式：`valid`或`same` [官方文档](https://keras.io/zh/layers/convolutional/#conv2d)

对卷积的描述假设卷积窗口的中心方块都是相邻的。默认步幅为1，也可以使用步幅大于1的卷积。比如上面5x5的特征图，进行步幅为2的卷积（无填充）：
![img](images/005-004.png)
步进卷积在实践中很少使用，大多时候步幅为1。

下面还需要对特征图进行采样，我们**不用**步幅，而通常在用`最大池化`(max-pooling)。前面例子中也提到了。每次MaxPooling2D层之后，特征图的尺寸都会减半。

如果用步幅为1的卷积来做的化，如下：


In [15]:
model_no_max_pool = models.Sequential()
model_no_max_pool.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model_no_max_pool.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_no_max_pool.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [16]:
model_no_max_pool.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 conv2d_4 (Conv2D)           (None, 24, 24, 64)        18496     
                                                                 
 conv2d_5 (Conv2D)           (None, 22, 22, 64)        36928     
                                                                 
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


上面一直用步幅为1的卷积进行运算，结果看最后一层的特征图每个样本有 22x22x64=30976 个元素，如果结果是要展开到一个512的dense层上的话，那将会是有 1580万个参数，这个数据太夸张了，会导致严重的过拟合。




简而言之，使用下采样的原因：
1. 减少特征图的元素个数
2. 让观察窗口越来越大（即窗口覆盖原始输入的比例越来越大）

**注意**:
1. MaxPooling不是下采样的唯一方法
2. 可以用步幅来实现
3. 也可以采用平均池化来实现
